# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import math

In [23]:
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
timestamp    8188 non-null object
id           8188 non-null int64
group        8188 non-null object
action       8188 non-null object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [10]:
df.id.nunique()

6328

In [12]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [63]:
# this was the beginning of my attempt
g = []
seen = set()
for index, thing in df.iterrows():
    if thing['id'] not in seen:
        seen.add(thing['id'])
    else:
        g.append(thing['id'])

In [85]:
# this will create an unordered set that only has the unduplicated ids for those who have either clicked or viewed
# you can do this pseudo math with sets where setA - setB = a new set which will only include from setA was not in B
clicks = set(df[df.action=='click']['id'].unique())
views = set(df[df.action=='view']['id'].unique())
len(clicks - views)

0

In [168]:
ctr = set(df[df['group']=='control']['id'].unique())
exp = set(df[df['group']=='experiment']['id'].unique())
print("Number in control group: {} \t Number in experimental group: {}".format(len(ctr), len(exp)))

Number in control group: 3332 	 Number in experimental group: 2996


In [166]:
# since we subtracted one from the other and they have the same numbers, that means there is no overlap
print(len(ctr - exp))
print(len(exp - ctr))

3332
2996


In [115]:
# another way this could have been done would be to join the two:
print(len(ctr & exp))

0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [96]:
# h0: the experimental homepage will have equal number of clicks as the control on average
# h1: the experimental homepage will have more clicks than the control on average
alpha = 0.5

In [186]:
df_ctr = df.loc[df['group']=='control']
df_exp = df.loc[df['group']=='experiment']

In [185]:
df_ctr

,timestamp,id,group,action
3,2016-09-24 19:59:02.646620,671993,control,view
9,2016-09-24 21:00:12.278374,560027,control,view
23,2016-09-25 00:25:14.141290,281985,control,view
24,2016-09-25 01:14:48.603202,407864,control,view
25,2016-09-25 02:16:11.046654,342984,control,view
26,2016-09-25 02:41:58.928919,716544,control,view
27,2016-09-25 02:52:43.844199,398892,control,view
28,2016-09-25 02:53:25.459874,398892,control,click
31,2016-09-25 03:50:39.942229,769969,control,view
32,2016-09-25 03:50:49.200721,462225,control,view


In [187]:
prcnt_ctr = (len(df_ctr[df_ctr['action']=='click']['id'].unique()) / len(df_ctr[df_ctr['action']=='view']['id'].unique())) * 100
prcnt_exp = (len(df_exp[df_exp['action']=='click']) / len(df_exp[df_exp['action']=='view'])) * 100
print(prcnt_ctr)
print(prcnt_exp)

27.971188475390157
30.97463284379172


In [173]:
ctr_val = []
for index, x in df_ctr['action'].items():
    ctr_val.append(boo(x))

In [151]:
exp_val = []
for index, x in df_exp['action'].items():
    exp_val.append(boo(x))

In [161]:
sum(x ==1 for x in ctr_val) / sum(x ==0 for x in ctr_val)    

0.2797118847539016

In [162]:
sum(x ==1 for x in exp_val) / sum(x ==0 for x in exp_val)   

0.3097463284379172

In [154]:
stats.ttest_ind(ctr_val, exp_val, equal_var=False)

Ttest_indResult(statistic=-1.9312441445072492, pvalue=0.05348777384439879)

##### I got this p-value wrong because I neglected to cleanse each dataframe of duplicate id's like we did in the first section. By doing this I was counting those who had clicked also as someone who had viewed and not clicked since we had records of both. <br>
##### Flatiron used a pivot table to accomplish isolating the values:

In [202]:
df['count'] = 1

control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

In [203]:
control.head()

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [204]:
experiment.head()

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0


In [208]:
print(len(experiment.click))
print(len(control.click))

2996
3332


In [207]:
stats.ttest_ind(control.click, experiment.click, equal_var=False)

Ttest_indResult(statistic=-2.615440020788211, pvalue=0.008932805628674201)

In [209]:
# in FI's solution, their p-value was half the value of mine. They used a homemade Welch T-test, so I'm not sure how
# to account for the discrepancy 
theirs = stats.ttest_ind(control.click, experiment.click, equal_var=False)[1] /2
theirs

0.004466402814337101

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [218]:
len(experiment.click) * (sum(control.click)/len(control.click))

838.0168067226891

In [219]:
sum(experiment.click)

928.0

In [230]:
rate = control.click.mean()

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [237]:
xbar = 928
mu = 838
n = len(experiment)
var = n * rate * (1-rate)
sigma = np.sqrt(var)
sigma

24.568547907005815

In [239]:
z = (xbar - mu) / sigma 
z

3.6632201602088235

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [240]:
1 - stats.norm.cdf(z)

0.00012453214564778214

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **It's a lot smaller, but both lead to the rejection of the null hypothesis**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.